In [1]:
import numpy as np
from scipy.linalg import eig
from quantum_channel_transformations import *


Choose an initial *pure* state of $|x^+\rangle$, corresponding density matrix $\rho_i=|x^+\rangle\!\langle x^+|$. And we want to rotate our initial state to a final density matrix of $\rho_f=|x^-\rangle\!\langle x^-|$. We do this by applying $\sigma_z\rho_i\sigma_z^\dagger$

In [2]:
x_plus = 1/np.sqrt(2) * np.array([1,1])
sigma_z = np.array([[1,0],[0,-1]])
density_matrix = np.outer(x_plus,x_plus.conjugate().T)

rotation = sigma_z @ density_matrix @ sigma_z.conjugate().T

Now we vectorise the density matrix and construct the superoperator that will perform $\sigma_z\rho_i\sigma_z^\dagger$

In the following the "left" (right acting) operator is $\sigma_z$, the "right" (left acting) operator is $\sigma_z^\dagger$

In [3]:
dm_vec = density_matrix_to_liouville(density_matrix)
print(dm_vec)

superoperator = left_right_super(sigma_z,sigma_z.conjugate().T)
print(superoperator)

# perform the rotation in liouville space
final_state = np.dot(superoperator,dm_vec)
final_state_unvectorised = density_matrix_to_hilbert(final_state)
print(final_state_unvectorised)

[0.5 0.5 0.5 0.5]
[[ 1  0  0  0]
 [ 0 -1  0  0]
 [ 0  0 -1  0]
 [ 0  0  0  1]]
[[ 0.5 -0.5]
 [-0.5  0.5]]


$$\rho_f^{jl}\equiv\epsilon(\rho_i) = \Lambda^{ijkl}\rho_i^{ij}$$

In [4]:
# need to express superoperator in rank 4 form
superoperator_reshaped = unvectorise_superoperator(superoperator)

choi_matrix = transform_superoperator_to_choi(superoperator_reshaped)

# formula as in wood
final_dm_choi = np.einsum('ijkl,ik->jl',choi_matrix,density_matrix)
print(final_dm_choi)

[[ 0.5 -0.5]
 [-0.5  0.5]]


In [5]:
# eigenvalues of choi matrix
choi_matrix_as_matrix = vectorise_superoperator(choi_matrix)
eigvals, eigvect = eig(choi_matrix_as_matrix)

print(eigvals,eigvect)



[2.+0.j 0.+0.j 0.+0.j 0.+0.j] [[ 0.70710678  0.70710678  0.          0.        ]
 [ 0.          0.          1.          0.        ]
 [ 0.          0.          0.          1.        ]
 [-0.70710678  0.70710678  0.          0.        ]]


In [6]:
coulmn_vec = eigvect[:,0]

coulmn_vec_reshaped = eigvals[0] * coulmn_vec.reshape(2,2)
print(coulmn_vec_reshaped)

[[ 1.41421356+0.j  0.        +0.j]
 [ 0.        +0.j -1.41421356+0.j]]
